In [1]:
import pandas as pd
import mysql.connector
import re
conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Admin@Yunus111",
                               database = "project_2")

cursor = conn.cursor()

In [2]:
df_product = pd.read_csv(r'C:\Users\HP\OneDrive\Desktop\dataset\Products.csv')
df_product.columns = df_product.columns.str.lower()
df_product.columns = df_product.columns.str.replace(" ","_")

def extract_numeric(value):
    return float(re.sub(r'[^\d.]', '', value))

# Apply function to the relevant columns
df_product['unit_cost_usd'] = df_product['unit_cost_usd'].apply(extract_numeric)
df_product['unit_price_usd'] = df_product['unit_price_usd'].apply(extract_numeric)

In [3]:
df_product.head()

,productkey,product_name,brand,color,unit_cost_usd,unit_price_usd,subcategorykey,subcategory,categorykey,category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,21.57,101,MP4&MP3,1,Audio


In [4]:
insert_product_values = '''
    INSERT INTO product (
        productkey, 
        product_name, 
        brand, 
        color, 
        unit_cost_usd, 
        unit_price_usd, 
        subcategorykey, 
        subcategory, 
        categorykey, 
        category
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        product_name=VALUES(product_name),
        brand=VALUES(brand),
        color=VALUES(color),
        unit_cost_usd=VALUES(unit_cost_usd),
        unit_price_usd=VALUES(unit_price_usd),
        subcategorykey=VALUES(subcategorykey),
        subcategory=VALUES(subcategory),
        categorykey=VALUES(categorykey),
        category=VALUES(category)
'''

# Insert data into the product table
for index, row in df_product.iterrows():
    cursor.execute(insert_product_values, (
        row['productkey'],
        row['product_name'],
        row['brand'],
        row['color'],
        row['unit_cost_usd'],
        row['unit_price_usd'],
        row['subcategorykey'],
        row['subcategory'],
        row['categorykey'],
        row['category']
    ))

# Commit the transaction
conn.commit()